In [2]:
from src.config import API_KEY, SECRET_KEY
paper = True

In [6]:
from datetime import datetime, timedelta
 
import alpaca
from alpaca.data.historical.option import OptionHistoricalDataClient, OptionLatestQuoteRequest
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestQuoteRequest
from alpaca.data.requests import StockLatestQuoteRequest

from alpaca.trading.client import TradingClient, GetAssetsRequest
from alpaca.trading.requests import GetOptionContractsRequest, LimitOrderRequest, MarketOrderRequest, GetOrdersRequest
from alpaca.trading.enums import AssetStatus, ContractType, OrderSide, OrderType, TimeInForce, QueryOrderStatus

In [3]:
trade_client = TradingClient(api_key=API_KEY, secret_key=SECRET_KEY, paper=paper)
stock_data_client = StockHistoricalDataClient(api_key=API_KEY, secret_key=SECRET_KEY)
option_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=SECRET_KEY)

In [8]:
req = StockLatestQuoteRequest(
    symbol_or_symbols='TSLA'
)
stock_data_client.get_stock_latest_quote(req)

{'TSLA': {   'ask_exchange': 'V',
     'ask_price': 302.0,
     'ask_size': 1.0,
     'bid_exchange': 'V',
     'bid_price': 295.3,
     'bid_size': 1.0,
     'conditions': ['R'],
     'symbol': 'TSLA',
     'tape': 'C',
     'timestamp': datetime.datetime(2025, 5, 9, 19, 59, 59, 412841, tzinfo=TzInfo(UTC))}}

In [4]:
from alpaca.data.live import StockDataStream




In [ ]:
# Create request parameters for the latest option trade
request_params = OptionLatestQuoteRequest(
	symbol="TSLA",  # Replace with the desired symbol
	expiration_date="2023-12-15",  # Replace with the desired expiration date
	strike_price=200.0,  # Replace with the desired strike price
	option_type="call"  # Replace with "call" or "put"
)

# Fetch the latest option trade
latest_trade = option_data_client.get_option_latest_trade(request_params)
latest_trade

In [ ]:
acct = trade_client.get_account()
# print(f"Options approved level: {acct.options_approved_level}")
# print(f"Options trading level: {acct.options_trading_level}")
# print(f"Options Buying power: {acct.options_buying_power}")
acct

In [ ]:
# set max options trading level, risk management
acct_config = trade_client.get_account_configurations()
print(f"max options trading level: {acct_config.max_options_trading_level}")

acct_config.max_options_trading_level = None
trade_client.set_account_configurations(acct_config)

acct_config = trade_client.get_account_configurations()
print(f"max options trading level: {acct_config.max_options_trading_level}")

In [ ]:
# active options to buy
req = GetAssetsRequest(
    status = AssetStatus.ACTIVE,
    attributes = "options_enabled"
)

options_enabled_underlyings = trade_client.get_all_assets(req)
print(f"Number of Underlyings with options: {len(options_enabled_underlyings)}")

symbol = 'TSLA'
test = trade_client.get_asset(symbol_or_asset_id=symbol)
test
# print(f"{symbol} has options? {'options_enabled' in test.attributes}")

In [ ]:
# account positions
positions = trade_client.get_all_positions()
for p in positions:
    print(p)

In [ ]:
today = datetime.now().date()
min_expiration = today + timedelta(days=14)
max_expiration = today + timedelta(days=90)

trade_req = StockLatestTradeRequest(symbol_or_symbols=symbol)
latest_trade = stock_data_client.get_stock_latest_trade(trade_req)
last_price = latest_trade[symbol].price

min_strike = str(round(last_price*1.02,2))

print(f"{symbol} last trade price: {last_price}")
print(f"Min exp: {min_expiration}")
print(f"Max exp: {max_expiration}")
print(f"Min strike: {min_strike}")

In [ ]:
# create market order
req = MarketOrderRequest(
    symbol='AAPL',
    qty=1,
    side=OrderSide.SELL,
    type=OrderType.MARKET,
    time_in_force=TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

# limit price request, buy when option reaches the limit price
limit_order = LimitOrderRequest(
    symbol='SPY',
    qty=1,
    side=OrderSide.BUY,
    time_in_force=TimeInForce.DAY, # will sit until timeInForce determines when to be filled
    limit_price=486.00
)
order = trade_client.submit_order(limit_order)

In [ ]:
# get unfilled orders to manage based off trading algorithm
req = GetOrdersRequest(
    status=QueryOrderStatus.OPEN
)
order = trade_client.get_orders(req)
# print(order)

for elem in order:
    trade_client.cancel_order_by_id(elem.id)

In [ ]:
# close position
symbol = positions[0].symbol
trade_client.close_position(
    symbol_or_asset_id=symbol
)